In [ ]:
import itertools as it
from iterdub import iterdub as ib
from iterpop import iterpop as ip
from keyname import keyname as kn
from matplotlib import pyplot as plt
import matplotlib
from nbmetalog import nbmetalog as nbm
import numpy as np
import pandas as pd
import patchworklib as pw
import pathlib
import random
from scipy import stats
import seaborn as sns
from slugify import slugify
import statsmodels.api as sm
import statsmodels.formula.api as smf
from teeplot import teeplot as tp
from tqdm import tqdm


In [ ]:
np.random.seed(1)
random.seed(1)


In [ ]:
from conduitpylib.viz import _get_defaults as cfg
from conduitpylib.viz import (
    get_notomoji_font_path,
    performance_semantics_scatterplot,
    performance_semantics_facetplot,
)
from conduitpylib.wrangle import retrieve_and_prepare_delta_dataframes


In [ ]:
nbm.print_metadata()


# Get Data


In [ ]:
def apply(df: pd.DataFrame) -> pd.DataFrame:
    row_distiller = lambda row: {
        k: v for k, v in row.items() if k in ("Num Nodes", "Num Processes")
    }
    df["Multiprocessing"] = df.apply(
        lambda row: {
            frozenset(
                {"Num Nodes": 1, "Num Processes": 2}.items()
            ): "Intranode",
            frozenset(
                {"Num Nodes": 2, "Num Processes": 2}.items()
            ): "Internode",
        }[frozenset(row_distiller(row).items())],
        axis=1,
    )
    return df


longitudinal, snapshot = retrieve_and_prepare_delta_dataframes(
    df_inlet_url="https://osf.io/4ys9v/download",
    df_outlet_url="https://osf.io/aum7w/download",
    apply=apply,
    treatment_column="Multiprocessing",
)


In [ ]:
for data in "longitudinal", "snapshot":
    for allocation, group in eval(data).groupby("Multiprocessing"):
        group_df = group.reset_index()
        tp.tee(
            performance_semantics_scatterplot,
            data=group_df,
            x="Simstep Period Inlet (ns)",
            y="Latency Simsteps Inlet",
            heat="% Msgs Lost",
            legend_font_name=get_notomoji_font_path(),
            title="__{}__".format(
                {
                    "Intranode": "⚁",
                    "Internode": "⚀⚀",
                }[allocation]
            ),
            batch_kwargs={
                "batch_size": 10,
                "progress_apply": tqdm,
            },
            bunching_smear_alpha=0.2,
            scatter_kwargs={
                "alpha": 0.4,
            },
            teeplot_outattrs={
                "allocation": slugify(allocation),
                "data": slugify(data),
            },
        )
        plt.show()


In [ ]:
for data in "longitudinal", "snapshot":
    plt.clf()
    group_df = group.reset_index()
    with pw.patched_axisgrid():
        facetplot = tp.tee(
            performance_semantics_facetplot,
            data=eval(data),
            x="Simstep Period Inlet (ns)",
            y="Latency Simsteps Inlet",
            hue="Multiprocessing",
            hue_order=["Intranode", "Internode"],
            legend_font_name=get_notomoji_font_path(),
            title="⚀⚀/⚁",
            teeplot_outattrs={
                "data": slugify(data),
            },
        )
        display(facetplot)
        plt.show()
